In [1]:
# Initialise Ray
import ray

In [6]:
ray.init(dashboard_host="0.0.0.0")

2020-11-29 03:08:57,753	INFO services.py:1090 -- View the Ray dashboard at http://172.17.0.3:8267


{'node_ip_address': '172.17.0.3',
 'raylet_ip_address': '172.17.0.3',
 'redis_address': '172.17.0.3:51858',
 'object_store_address': '/tmp/ray/session_2020-11-29_03-08-57_170283_1377/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-11-29_03-08-57_170283_1377/sockets/raylet',
 'webui_url': '172.17.0.3:8267',
 'session_dir': '/tmp/ray/session_2020-11-29_03-08-57_170283_1377',
 'metrics_export_port': 57434,
 'node_id': '8efb22da665d033a6628862fc077109b9e048a0e'}

# Ray Tune Quickstart

In [7]:
from ray import tune

In [8]:
def objective(step, alpha, beta):
    return (0.1 + alpha * step / 100)**(-1) + beta * 0.1


def training_function(config):
    # Hyperparameters
    alpha, beta = config["alpha"], config["beta"]
    for step in range(10):
        # Iterative training function - can be any arbitrary training procedure.
        intermediate_score = objective(step, alpha, beta)
        # Feed the score back back to Tune.
        tune.report(mean_loss=intermediate_score)


analysis = tune.run(
    training_function,
    config={
        "alpha": tune.grid_search([0.001, 0.01, 0.1]),
        "beta": tune.choice([1, 2, 3])
    })

print("Best config: ", analysis.get_best_config(
    metric="mean_loss", mode="min"))

# Get a dataframe for analyzing trial results.
df = analysis.results_df

2020-11-29 03:09:37,858	WARNING function_runner.py:539 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2020-11-29 03:09:37,876	WARNING tune.py:401 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.
2020-11-29 03:09:37,883	INFO logger.py:200 -- pip install 'ray[tune]' to see TensorBoard files.
2020-11-29 03:09:37,884	WARNING logger.py:342 -- Could not instantiate TBXLogger: No module named 'tensorboardX'.
2020-11-29 03:09:37,885	ERROR syncer.py:63 -- Log sync requires rsync to be installed.


Trial name,status,loc,alpha,beta
training_function_50945_00000,RUNNING,,0.001,1


Result for training_function_50945_00000:
  date: 2020-11-29_03-09-38
  done: false
  experiment_id: ad73b485badf4657ac391494cda02ff9
  experiment_tag: 0_alpha=0.001,beta=1
  hostname: 0a1efe644149
  iterations_since_restore: 1
  mean_loss: 10.1
  neg_mean_loss: -10.1
  node_ip: 172.17.0.3
  pid: 2260
  time_since_restore: 0.00016951560974121094
  time_this_iter_s: 0.00016951560974121094
  time_total_s: 0.00016951560974121094
  timestamp: 1606619378
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: '50945_00000'
  
Result for training_function_50945_00002:
  date: 2020-11-29_03-09-38
  done: false
  experiment_id: da1e90311a684d03adf009a5642d1835
  experiment_tag: 2_alpha=0.1,beta=3
  hostname: 0a1efe644149
  iterations_since_restore: 1
  mean_loss: 10.3
  neg_mean_loss: -10.3
  node_ip: 172.17.0.3
  pid: 2259
  time_since_restore: 0.0002052783966064453
  time_this_iter_s: 0.0002052783966064453
  time_total_s: 0.0002052783966064453
  timestamp: 1606619378
  timesteps_sin

Trial name,status,loc,alpha,beta,loss,iter,total time (s),neg_mean_loss
training_function_50945_00000,TERMINATED,,0.001,1,10.091,10,0.0454204,-10.091
training_function_50945_00001,TERMINATED,,0.01,3,10.2108,10,0.0565863,-10.2108
training_function_50945_00002,TERMINATED,,0.1,3,9.47431,10,0.0564847,-9.47431


2020-11-29 03:09:38,605	INFO tune.py:439 -- Total run time: 0.75 seconds (0.71 seconds for the tuning loop).


Best config:  {'alpha': 0.1, 'beta': 3}


# Shutdown

In [9]:
ray.shutdown()